# Gap Data Engineering Framework - CV Code Along with TensorFlow

## Sign-Language Dataset

We will do the following in this tutorial:

    1. Install the VISION module of the Gap Framework.
    2. Install TensorFlow for Machine Learning Model.
    3. Use the VISION module to preprocess the image data and make min-batches for training.
    4. Train the model.

### Install Vision Module

We start by importing the Gap Vision module

In [1]:
# import the Gap Vision module
from vision import Images

### Install TensorFlow

Next we import the TensorFlow package

In [2]:
# import Tensorflow
# Importing Tensorflow
import tensorflow as tf
print( tf.__version__ )

1.8.0


### Data Source

Github Public Repository - https://github.com/EvilPort2/Sign-Language

<i>"The first thing I did was, I created 10 gesture samples using OpenCV. For each gesture I captured 1200 images which were 50x50 pixels. All theses images were in grayscale which is stored in the gestures/ folder. The gestures/0/ folder contains 1200 blank images which signify "none" gesture. Also I realised that keeping this category increased my model's accuracy to 99% from a laughable 82%."</i>

Let's go to the downloaded images and see what's there. Let's first check that we are in the correct directory.

In [3]:
pwd

"C:\\Users\\'\\Desktop\\Training\\AITraining\\Intermediate\\Machine Learning\\sign-lang"

You will see under the gestures directory, we have folders 0 through 26. Folder 0 is the none image, while folders 1 through 26 have images for the letters A..Z, respectively.

Let's look under one of the folders (i.e., folder 1 for the letter A):

<img src='gestures.jpg'/>

### Create Images Collections

We will use the Images class to create a collection for each set (i.e., letter in alphabet) of training images.

To load and process the images, we do:
    1. Each subfolder under gestures corresponds to a letter, where the subfolder name is the label.
    2. Get a list of all files (images) under each subfolder.
    3. Create an Images collection per subfolder, where the files in the subfolder are the images, and the subfolder name is the label.

For the config(uration) options:
    - Process as grayscale images
    - To reduce storage space, we are not going to store the raw pixel data.


In [4]:
import os

# A list of the Images object for each subfolder (i.e.,letter)
collections = []

total_time = 0

# There are 27 sets of images under gestures, each in its own subfolder
for label in os.listdir("gestures"):
    subfolder = "gestures/" + label + "/"
    
    # Let's get the list of images under the subfolder for this set.
    files = os.listdir(subfolder)
    
    # We need the relative path to each file, we will use a generator for this purpose
    files = [subfolder + file for file in files]
    
    # Let's create now the Images object for the subfolder of images
    images = Images(files, labels=int(label), name='tmp' + label, config=['grayscale', 'flatten', 'nostore'])
    
    collections.append(images)
    
    # time metrics
    print(label, images.time)
    total_time += images.time

print("Total Time:", total_time)

0 7.052011013031006
1 7.190010070800781
10 8.081011533737183
11 7.5000104904174805
12 7.570010423660278
13 7.662010908126831
14 12.601017713546753
15 6.170008659362793
16 7.514014720916748
17 5.990008354187012
18 5.862013339996338
19 6.27000880241394
2 9.427016258239746
20 5.75000786781311
21 6.360008955001831
22 5.7120091915130615
23 6.340008735656738
24 4.982008218765259
25 5.334014892578125
26 6.292011260986328
3 5.78000807762146
4 6.112008571624756
5 4.940006971359253
6 5.462008953094482
7 4.792011976242065
8 4.700006723403931
9 4.620006561279297
Total Time: 176.0652792453766


### Splitting into Training and Test

Gap v0.9 (alpha) doesn't have a Dataset class yet for Images, so we are going to prepare our training data with a little brute force. Don't worry, that will change soon.

    - We use the split property to split each collection into training and test data
    - We use the list extend() method to build our list w/o copying any data

In [5]:
# We will collect all the training sets from each collection into a list
X_train = []
X_test  = []
Y_train = []
Y_test  = []

# 80% of the images will be used for training (note that the list is randomized)
for images in collections:
    images.split = 0.2, 42

    # When used as a getter, the split property will return the training / test data and labels the same as the sci-learn
    # procedure train_test_split()
    x_train, x_test, y_train, y_test = images.split
    
    # Let's concatenate (without copying) the split of each collection to a single contiguous list
    X_train.extend(x_train)
    X_test.extend(x_test)
    Y_train.extend(y_train)
    Y_test.extend(y_test)

Some more brute force correction. We will change the python list for the collections to a numpy array

In [6]:
import numpy as np
X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)
X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)

One last step, for this Tensorflow model, we need to convert our labels into a one hot encoding (dummy variable conversion).

In [7]:
def convert_labels_to_one_hot_encoding(Y, C):
    """ This function will do the reshape and conversion (from Coursera)"""
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

# Let's do the conversion
Y_train = convert_labels_to_one_hot_encoding(Y_train, 27)
Y_test  = convert_labels_to_one_hot_encoding(Y_test, 27)

# Let's print the shape of our labels
print( Y_train.shape )

(25920, 27)


## Build a Graph

In [13]:
# Let's first reset our graph, so our neural network components are all declared within the same graph
from tensorflow.python.framework import ops
ops.reset_default_graph() 

### Placeholders

In [14]:
X = tf.placeholder(tf.float32, shape=[2500, None])
Y = tf.placeholder(tf.float32, shape=[27, None])
D = tf.placeholder(tf.float32, [])
L = tf.placeholder(tf.float32, [])

### Input Layer

Let's now design our input layer. We need two things: weights and biases.

Each input feature (pixel) will need a weight (which our model will learning during training). The weight is multipled against the value of the input (pixel), which we symbolically represent as Wx.

Each output from the layer will need a bias (which our model will learning during training). The bias is added to the result of the weight multipled by the pixel value (Wx).

Let's create two Tensorflow variables for our weights and biases. The weights (which we call W) will need to be a 2D matrix. The rows are the number of inputs, which is 2500 and the columns the number of outputs to the hidden layer, which will be 64.

The bias will be a vector of size 64 (one for each output).

We need to initialize our weights and biases to some initial value. We will initialize the weights using a random value initializer (Xavier) and initialize the biases to zero.

In [15]:
tf.set_random_seed(1)   # Set the same seed to get the same initialization as in this demo.

# The weights for the input layer
W1 = tf.get_variable("W1", [64, 2500], initializer=tf.contrib.layers.xavier_initializer(seed=1))

# The bias for the output from the input layer
b1 = tf.get_variable("b1", [64, 1], initializer=tf.zeros_initializer())

In [16]:
# The first layer (input layer)
Z1 = tf.add(tf.matmul(W1, X), b1)

# Let's add the activation function to the output signal from the first layer
A1 = tf.nn.relu(Z1)

### First Hidden Layer

The first hidden layer will have 64 inputs (outputs from input layer) and 32 outputs. Each input will need a weight and each output a bias (which we will train). Each output will be passed through the linear rectifier unit (RELU) activation function.

We will initialize the weights using a random value initializer (Xavier) and initialize the biases to zero.

In [17]:
W2 = tf.get_variable("W2", [32, 64], initializer=tf.contrib.layers.xavier_initializer(seed=1))
b2 = tf.get_variable("b2", [32, 1], initializer=tf.zeros_initializer())

Let's construct the first hidden layer

    Create a node that will multiply the weights (W2) against the outputs of the input layer (A1).
    Create a node that adds the bias to the above node (W2 * A1)
    Pass the outputs from the (first) hidden layer through a dropout layer
    Pass the outputs from the dropout layer through a RELU activation function


In [18]:
# The second layer (first hidden layer)
Z2 = tf.add(tf.matmul(W2, A1), b2) 

# Let's add the dropout layer to the output signal from the second layer
D2 = tf.nn.dropout(Z2, keep_prob=D)

# Let's add the activation function to the output signal from the dropout layer
A2 = tf.nn.relu(D2)

### Second Hidden Layer

The second hidden layer will have 32 inputs (outputs from first hidden layer) and 20 outputs. Each input will need a weight and each output a bias (which we will train). Each output will be passed through the linear rectifier unit (RELU) activation function.

We will initialize the weights using a random value initializer (Xavier) and initialize the biases to zero.

In [20]:
W3 = tf.get_variable("W3", [20, 32], initializer=tf.contrib.layers.xavier_initializer(seed=1))
b3 = tf.get_variable("b3", [20, 1], initializer=tf.zeros_initializer())

Let's construct the second hidden layer.

    Create a node that will multiply the weights (W3) against the outputs of the first hidden layer (A2).
    Create a node that adds the bias to the above node (W3 * A2)
    Pass the outputs from the second hidden layer through a RELU activation function

In [21]:
# The third layer (second hidden layer)
Z3 = tf.add(tf.matmul(W3, A2), b3) 

# Let's add the activation function to the output signal from the third layer
A3 = tf.nn.relu(Z3)

### Output Layer

The output layer will have 20 inputs (outputs from the second hidden layer) and 27 outputs (one for each letter and None). Each input will need a weight and each output a bias (which we will train). The 27 outputs will be passed through a softmax activation function.

We will initialize the weights using a random value initializer (Xavier) and initialize the biases to zero.

In [22]:
W4 = tf.get_variable("W4", [27, 20], initializer=tf.contrib.layers.xavier_initializer(seed=1))
b4 = tf.get_variable("b4", [27, 1], initializer=tf.zeros_initializer())

Let's construct the output layer.

    Create a node that will multiply the weights (W4) against the outputs of the second hidden layer (A3).
    Create a node that adds the bias to the above node (W4 * A3)
    Pass the outputs from the output layer through a SOFTMAX squashing function (done by the optimizer)



In [23]:
# The fourth layer (output layer)
Z4 = tf.add(tf.matmul(W4, A3), b4)

### Optimizer

Now its time to design our optimizer. Let's start by designing our cost function. We will use the mean value of the softmax cross entropy between the predicted labels and actual labels. This is what we want to reduce on each batch (aka the cost).

In [25]:
# to fit the tensorflow requirement for tf.nn.softmax_cross_entropy_with_logits(...,...)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=tf.transpose(Z4), labels=tf.transpose(Y)))

Let's design our optimizer. This is the method that adjusts the values of the weights and biases, based on minizing the cost value during training.

We also need to set a learning rate. This is multiplied against the gradient calculation. It's used to prevent huge swings in setting weights which can result in either converging at a local (instead of global) optima, or not converging at all (infinite gradient). We will set the learning rate when we run the graph using the placeholder L.


In [26]:
# The learning rate for Gradient Descent algorithm
optimizer = tf.train.GradientDescentOptimizer(L).minimize(cost)

### Run the Graph

In [27]:
init = tf.global_variables_initializer()

#### Hyperparameter Tuning

In [46]:
import time

epochs = 25                                    # run a 25 epochs
batch_size = 100                               # for each epoch, train in batches of 100 images
number_of_images = len(X_train)                # number of images in training data

# Feed Dictionary Parameters
keep_prob = 0.9                                # percent of outputs to keep in dropout layer
learning_rate = 0.004                          # the learning rate for graident descent

In [47]:
def train():
    start = time.time()

    with tf.Session() as sess:
        # Initialize the variables
        sess.run(init)
        
        # number of batches in an epoch
        batches = number_of_images // batch_size

        # run our training data through the neural network for each epoch
        for epoch in range(epochs):

          epoch_cost = 0

          # Run the training data through the neural network
          for batch in range(batches):

              # Calculate the start and end indices for the next batch
              begin = (batch * batch_size)
              end   = (batch * batch_size) + batch_size


              # Get the next sequential batch from the training data
              batch_xs, batch_ys = X_train[begin:end], Y_train[begin:end]

              # Feed this batch through the neural network.
              _, batch_cost = sess.run([optimizer, cost], feed_dict={X: batch_xs.T, Y: batch_ys.T, D: keep_prob, L: learning_rate})

              epoch_cost += batch_cost

          print("Epoch: ", epoch, epoch_cost / batches)

        end = time.time()

        print("Training Time:", end - start)

        # Test the Model

        # Let's select the highest percent from the softmax output per image as the prediction.
        prediction = tf.equal(tf.argmax(Z4), tf.argmax(Y))

        # Let's create another node for calculating the accuracy
        accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

        # Now let's run our trainingt images through the model to calculate our accuracy during training
        # Note how we set the keep percent for the dropout rate to 1.0 (no dropout) when we are evaluating the accuracy.
        print ("Train Accuracy:", accuracy.eval({X: X_train.T, Y: Y_train.T, D: 1.0}))

        # Now let's run our test images through the model to calculate our accuracy on the test data
        print ("Test Accuracy:", accuracy.eval({X: X_test.T, Y: Y_test.T, D: 1.0}))
        
train()

Epoch:  0 2.6024440230089727
Epoch:  1 1.8829405013773892
Epoch:  2 1.3729867978330745
Epoch:  3 1.0144998312399194
Epoch:  4 0.8078569689600164
Epoch:  5 0.64922806351629
Epoch:  6 0.5507119966663679
Epoch:  7 0.4809992052542772
Epoch:  8 0.42571273187901754
Epoch:  9 0.3855076012593554
Epoch:  10 0.35134849454991834
Epoch:  11 0.3184934920634945
Epoch:  12 0.2944728962386667
Epoch:  13 0.272749159747598
Epoch:  14 0.2541119466575301
Epoch:  15 0.2332139004697901
Epoch:  16 0.22576703361152856
Epoch:  17 0.20729639536510563
Epoch:  18 0.19759646106986006
Epoch:  19 0.18437778887529516
Epoch:  20 0.1767506640670555
Epoch:  21 0.16767139829026398
Epoch:  22 0.15965445201959044
Epoch:  23 0.1516163613499793
Epoch:  24 0.14672508483939592
Training Time: 110.57516121864319
Train Accuracy: 0.73383486
Test Accuracy: 0.7337963
